In [1]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import keras_tuner as kt
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
with open(r'.\Reviews_Dataset_Splits\X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\X_val.pkl', 'rb') as f:
    X_val = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_val.pkl', 'rb') as f:
    y_val = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

y_train = keras.utils.to_categorical(y_train-1, num_classes=5)
y_val = keras.utils.to_categorical(y_val-1, num_classes=5)
y_test = keras.utils.to_categorical(y_test-1, num_classes=5)

with open(r'.\embeddingMatrix_Reviews.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

num_tokens = len(embedding_matrix) # total vocabulary +1 or length of embedding matrix
embedding_dim = 300 # dimension of the vector of a single word
MAX_REVIEW_LEN = 250 # maximum words in a review
num_classes = 5

In [3]:
def build_model_pd(hp, test_optimizers, test_initializers, test_regularizer, regularizer_choice, test_learning_rate,
                test_activations):
    embedding_layer = keras.layers.Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        input_length=MAX_REVIEW_LEN,
        trainable=True,
        mask_zero=True)
    
    model = keras.Sequential()
    model.add(embedding_layer)

    if test_activations:
        activation_choice = hp.Choice('activation', ['softplus', 'softsign', 'relu', 'tanh'])
        activation = tf.keras.activations.get(activation_choice)
    else:
        activation = 'tanh'  # Default activation

    if test_initializers:
        initializer_choice = hp.Choice('kernel_initializer', ['glorot_uniform', 'he_uniform', 'random_uniform'])
        kernel_initializer = tf.keras.initializers.get(initializer_choice)
    else:
        kernel_initializer = 'he_uniform'

    if test_regularizer:
        if regularizer_choice == 'l1':
            kernel_regularizer = keras.regularizers.L1(l1=hp.Choice('l1_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l2':
            kernel_regularizer = keras.regularizers.L2(l2=hp.Choice('l2_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l1_l2':
            kernel_regularizer = keras.regularizers.L1L2(l1=hp.Choice('l1_l2_l1_factor', [1e-4, 1e-2]),
                                    l2=hp.Choice('l1_l2_l2_factor', [1e-4, 1e-2]))
    else:
        kernel_regularizer = None

    model.add(keras.layers.LSTM(64, activation=activation, return_sequences=True))
    model.add(keras.layers.LSTM(128, activation=activation, return_sequences=True))
    model.add(keras.layers.LSTM(64, activation=activation))

    model.add(layers.Dense(num_classes, activation='softmax'))

    # Optimizer
    if test_optimizers:
        optimizer = hp.Choice('optimizer', ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam'])
    else:
        if test_learning_rate:
            learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])
        else:
            learning_rate = 0.001
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
NUM_EPOCHS = 25
BATCH_SIZE = 128

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f'KerasTuner_Logs/RNN/RNN_V2_3Layer_Optimizer_{now}'

# Callbacks
tensorboard = TensorBoard(log_dir=f'TensorBoard_Logs/RNN/RNN_V2_3Layer_Optimizer_{now}')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

tuner = kt.GridSearch(
    lambda hp: build_model_pd(hp, test_optimizers=True, test_initializers=False, test_regularizer=False, 
                           regularizer_choice='l2', test_learning_rate=False, test_activations=False),
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=None,
    executions_per_trial=1,
    directory=directory,
    project_name='Reviews_Classification')

In [26]:
def build_model_pd2(hp, test_optimizers, test_initializers, test_regularizer, regularizer_choice, test_learning_rate,
                   test_activations, test_batch_norm):
    embedding_layer = keras.layers.Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        input_length=MAX_REVIEW_LEN,
        trainable=True,
        mask_zero=True)
   
    model = keras.Sequential()
    model.add(embedding_layer)
    
    if test_activations:
        activation_choice = hp.Choice('activation', ['softplus', 'softsign', 'relu', 'tanh'])
        activation = tf.keras.activations.get(activation_choice)
    else:
        activation = 'tanh'
    
    if test_initializers:
        initializer_choice = hp.Choice('kernel_initializer', ['glorot_uniform', 'he_uniform', 'random_uniform'])
        kernel_initializer = tf.keras.initializers.get(initializer_choice)
    else:
        kernel_initializer = 'he_uniform'
    
    if test_regularizer:
        if regularizer_choice == 'l1':
            kernel_regularizer = keras.regularizers.L1(l1=hp.Choice('l1_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l2':
            kernel_regularizer = keras.regularizers.L2(l2=hp.Choice('l2_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l1_l2':
            kernel_regularizer = keras.regularizers.L1L2(l1=hp.Choice('l1_l2_l1_factor', [1e-4, 1e-2]),
                                    l2=hp.Choice('l1_l2_l2_factor', [1e-4, 1e-2]))
    else:
        kernel_regularizer = None

    # Function to add an lstm block
    def add_lstm_block(layer, units, dropout_rate=0.2, ret_sequences = False):
        model.add(layers.LSTM(units=units, activation=activation, kernel_initializer=kernel_initializer, 
                              kernel_regularizer=kernel_regularizer, return_sequences=ret_sequences))
        if test_batch_norm:
            use_bn = hp.Boolean(f'batch_norm_after_lstm_{layer}')
            if use_bn:
                model.add(layers.BatchNormalization())
        if dropout_rate > 0:
            model.add(layers.Dropout(rate=dropout_rate))

    # Add convolutional blocks
    add_lstm_block(1, 64, dropout_rate=0, ret_sequences = True)
    add_lstm_block(2, 128, dropout_rate=0, ret_sequences = True)
    add_lstm_block(3, 64, dropout_rate=0, ret_sequences = False)

    model.add(layers.Dense(num_classes, activation='softmax'))
    
    # Optimizer
    if test_optimizers:
        optimizer = hp.Choice('optimizer', ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam'])
    else:
        if test_learning_rate:
            learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])
        else:
            learning_rate = 0.001
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [27]:
NUM_EPOCHS = 25
BATCH_SIZE = 128

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f'KerasTuner_Logs/RNN/RNN_V7_3Layer_BatchNorm_{now}'

# Callbacks
tensorboard = TensorBoard(log_dir=f'TensorBoard_Logs/RNN/RNN_V7_3Layer_BatchNorm_{now}')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

tuner = kt.GridSearch(
    lambda hp: build_model_pd2(hp, test_optimizers=False, test_initializers=False, test_regularizer=False, 
                           regularizer_choice='l1_l2', test_learning_rate=False, test_activations=False, test_batch_norm=True),
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=None,
    executions_per_trial=1,
    directory=directory,
    project_name='Reviews_Classification')

In [31]:
def build_model_do(hp, use_dropout):
    embedding_layer = keras.layers.Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        input_length=MAX_REVIEW_LEN,
        trainable=True,
        mask_zero=True)
   
    model = keras.Sequential()
    model.add(embedding_layer)

    activation = 'tanh'  # Default activation
    kernel_initializer = 'he_uniform'

    model.add(layers.LSTM(64, activation=activation, kernel_initializer=kernel_initializer, return_sequences=True))
    model.add(layers.BatchNormalization())
    if use_dropout:
            model.add(layers.Dropout(rate=hp.Choice(f'dropout_rate_1', [0.0, 0.2, 0.4])))

    model.add(layers.LSTM(128, activation=activation, kernel_initializer=kernel_initializer, return_sequences=True))
    if use_dropout:
            model.add(layers.Dropout(rate=hp.Choice(f'dropout_rate_2', [0.0, 0.2, 0.4])))

    model.add(layers.LSTM(64, activation=activation, kernel_initializer=kernel_initializer, return_sequences=False))
    if use_dropout:
            model.add(layers.Dropout(rate=hp.Choice(f'dropout_rate_3', [0.0, 0.2, 0.4])))

    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [32]:
NUM_EPOCHS = 15
BATCH_SIZE = 128

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f'KerasTuner_Logs/RNN/RNN_V8_3Layer_DropOut_{now}'

# Callbacks
tensorboard = TensorBoard(log_dir=f'TensorBoard_Logs/RNN/RNN_V8_3Layer_DropOut_{now}')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

tuner = kt.GridSearch(
    lambda hp: build_model_do(hp, use_dropout=True),
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=None,
    executions_per_trial=1,
    directory=directory,
    project_name='Reviews_Classification')

In [33]:
tuner.search(x=X_train,
             y=y_train,
             verbose=1,
             epochs=NUM_EPOCHS,
             batch_size=BATCH_SIZE,
             callbacks=[tensorboard, early_stopping],
             validation_data=(X_val, y_val))

Trial 27 Complete [00h 03m 48s]
val_loss: 1.100034475326538

Best val_loss So Far: 1.0505949258804321
Total elapsed time: 01h 42m 31s


In [34]:
tuner.get_best_hyperparameters()[0].values

{'dropout_rate_1': 0.0, 'dropout_rate_2': 0.0, 'dropout_rate_3': 0.0}

In [35]:
tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 300)          16058400  
                                                                 
 lstm (LSTM)                 (None, 250, 64)           93440     
                                                                 
 batch_normalization (BatchN  (None, 250, 64)          256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 250, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 250, 128)          98816     
                                                                 
 dropout_1 (Dropout)         (None, 250, 128)          0         
                                                        

In [21]:
tuner.results_summary()

Results summary
Results in KerasTuner_Logs/CNN/CNN_V5_3Layer_RegularizerL2_2024-07-13_13-28-40\Reviews_Classification
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0000 summary
Hyperparameters:
l2_factor: 0.0001
Score: 1.1286063194274902

Trial 0001 summary
Hyperparameters:
l2_factor: 0.01
Score: 1.6093945503234863
